In [1]:
import numpy as np

In [2]:
def get_model(save_name='Xception.hdf5', image_shape=(299, 299, 3)):
    import os
    new_model = not os.path.exists(save_name)
    if new_model:
        from keras.applications.xception import Xception
        model = Xception(
            include_top=False, weights='imagenet',
            input_shape=image_shape, pooling='avg'
        )
        model.save(save_name)
    else:
        from keras.models import load_model
        model = load_model(save_name)
    return model, new_model

In [3]:
initial_model, new_model = get_model()
if new_model:
    print('new model')
else:
    print('old model')

Using TensorFlow backend.


old model


/usr/lib/python3.6/site-packages/keras/models.py:245: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
def get_data(image_shape=(299, 299, 3)):
    from skimage.transform import resize
    from skimage.io import imread
    import os
    images = []
    labels = []
    with open('public_data/00_input/train/gt.csv') as navigate_file:
        for i, line in enumerate(navigate_file):
            if i == 0:
                continue
            image_name, label = line.split(',')                
            images.append(resize(
                imread(os.path.join(
                    'public_data/00_input/train/images/',
                    image_name
                )),
                image_shape,
                mode='reflect'
            ))
            labels.append(int(label))
    return np.array(images), np.array(labels)

In [5]:
x, y = get_data()

In [6]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import Nadam
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [7]:
num_classes = len(set(y))
y = np.eye(num_classes)[y]
train_x, test_x, train_y, test_y = train_test_split(
    x, y, test_size=0.3, random_state=42
)

In [8]:
data_generator = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
)

In [9]:
regularization_lambda = 1e-3

last = initial_model.output

prediction = Dense(
    num_classes, activation='softmax',
    kernel_regularizer=regularizers.l2(regularization_lambda)
)(last)

model = Model(initial_model.input, prediction)

In [10]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=1e-3, decay=1e-3),
    metrics=['categorical_accuracy']
)

In [ ]:
class MyClallback:
    def __init__(self, model, directory):
        self._model = model
        self._directory = directory
        self._counter = 1

    def set_model(self, model):
        pass

    def set_params(self, params):
        pass

    def on_train_begin(self, logs):
        pass

    def on_train_end(self, logs):
        pass

    def on_epoch_begin(self, epoch, logs):
        pass

    def on_epoch_end(self, epoch, logs):
        self._model.save(
            self._directory + '/' + str(self._counter) +
            '_loss:_' + str(logs['loss']) +
            '_accuracy:_' + str(logs['categorical_accuracy']) +
            '_val_loss:_' + str(logs['val_loss']) +
            '_val_accuracy:_' + str(logs['val_categorical_accuracy']) +
            '.npz'
        )
        self._counter += 1

    def on_batch_begin(self, batch, logs):
        pass

    def on_batch_end(self, batch, logs):
        pass

In [ ]:
batch_size = 4
model.fit_generator(
    data_generator.flow(train_x, train_y, batch_size=batch_size),
    steps_per_epoch=int(len(train_y) / batch_size), epochs=300,
    callbacks=[MyClallback(model, 'models')],
    validation_data=(test_x, test_y)
)

Epoch 1/300
437/437 [==============================] - 228s - loss: 3.9477 - categorical_accuracy: 0.0280 - val_loss: 5.6807 - val_categorical_accuracy: 0.0320
Epoch 2/300
437/437 [==============================] - 213s - loss: 3.7466 - categorical_accuracy: 0.0452 - val_loss: 3.7232 - val_categorical_accuracy: 0.0520
Epoch 3/300
437/437 [==============================] - 211s - loss: 3.4736 - categorical_accuracy: 0.0801 - val_loss: 3.5721 - val_categorical_accuracy: 0.0880
Epoch 4/300
437/437 [==============================] - 217s - loss: 3.1381 - categorical_accuracy: 0.1236 - val_loss: 3.0063 - val_categorical_accuracy: 0.1573
Epoch 5/300
437/437 [==============================] - 208s - loss: 2.7418 - categorical_accuracy: 0.1974 - val_loss: 2.6565 - val_categorical_accuracy: 0.2107
Epoch 6/300
437/437 [==============================] - 209s - loss: 2.3704 - categorical_accuracy: 0.2946 - val_loss: 2.1080 - val_categorical_accuracy: 0.3587
Epoch 7/300
437/437 [===================

In [ ]:
model.save('0.72_score_model.hdh5')